In [ ]:
from pony.orm import *
import geocoder
import requests
import json

In [ ]:
from bail.db import DB, Case, GeocodedAddress
db = DB()

In [ ]:
data = select(c for c in Case if c.address != None)

In [ ]:
with requests.Session() as session:
    def geocode(x):
        if not x or x == "":
            return None
        existing = GeocodedAddress.select(lambda a: x == a.address)
        if existing:
            print(f"{x}: existing!")
        else:
            g = geocoder.osm(x, session=session)
            if g.ok:
                GeocodedAddress(
                    address=x,
                    latitude=g.latlng[0],
                    longitude=g.latlng[1],
                )
                flush()
                print(f"{x}: {g.latlng}")

    for c in data:
        geocode(c.address)

In [ ]:
addresses = [{'address': a.address, 'longitude': a.longitude, 'latitude': a.latitude} for a in GeocodedAddress.select()]

In [ ]:
out_file = open("geocode.json", "w") 
json.dump(addresses, out_file)

In [ ]:
GeocodedAddress.select().count()